## Export ONNX

In [1]:
import onnx
import torch
import onnxruntime
import numpy as np
import torch.nn as nn
import torch.nn.init as init


C:\Users\USER\anaconda3\envs\preparation\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class ReviewClassifier(nn.Module):
    def __init__(self):
        super(ReviewClassifier, self).__init__()
        self.fc_1 = nn.Linear(768 * 2, 512)
        self.fc_2 = nn.Linear(512, 512)
        self.fc_3 = nn.Linear(512, 3)

        self.relu = nn.ReLU()
        self.leaky = nn.LeakyReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        z = self.leaky(self.fc_1(x))
        z = self.dropout(z)
        z = self.leaky(self.fc_2(z))
        z = self.dropout(z)
        # z = self.leaky(self.fc_2(z))
        # z = self.dropout(z)
        z = self.fc_3(z)
        return z

In [3]:
model_path = "Architecture-Score_Classifier-240116_0623-021-8320-8502.pt"
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model = ReviewClassifier()
model = torch.load(model_path)
# model.load_state_dict(weight)
batch_size = 1
x = torch.randn(batch_size,1,1,768 * 2, requires_grad = True)
torch_out = model(x)

In [4]:
save_model_name = "Architecture-Score_Classifier.onnx"
torch.onnx.export(model,               # 실행될 모델
                  x,                         # 모델 입력값 (튜플 또는 여러 입력값들도 가능)
                  save_model_name,   # 모델 저장 경로 (파일 또는 파일과 유사한 객체 모두 가능)
                  export_params=True,        # 모델 파일 안에 학습된 모델 가중치를 저장할지의 여부
                  opset_version=10,          # 모델을 변환할 때 사용할 ONNX 버전
                  do_constant_folding=True,  # 최적화시 상수폴딩을 사용할지의 여부
                  input_names = ['input'],   # 모델의 입력값을 가리키는 이름
                  output_names = ['output'], # 모델의 출력값을 가리키는 이름
                  dynamic_axes={'input' : {0 : 'batch_size'},    # 가변적인 길이를 가진 차원
                                'output' : {0 : 'batch_size'}})

## Load ONNX and test

In [5]:
save_model_name = "Architecture-Score_Classifier.onnx"
onnx_model = onnx.load(save_model_name)
onnx.checker.check_model(onnx_model)

In [6]:
ort_session = onnxruntime.InferenceSession(save_model_name)

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# ONNX 런타임에서 계산된 결과값
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

In [7]:
# x = input 값
batch_size = 1
x = torch.randn(batch_size,1,1,768 * 2, requires_grad = True)
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)
output_y = ort_outs[0]

In [8]:
print(output_y)

[[[[-6.611784   2.6353707  0.7933747]]]]
